In [1]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet', 'val': 'data/val-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/Kanakmi/mental-disorders/" + splits["train"])

c:\Users\kiera\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df.head()

,text,label
0,My father - all of my life - has shifted betwe...,0
1,I have health anxiety where I go to the doctor...,3
2,I was thinking about the differences between B...,0
3,Let me preface this by saying that I promise I...,2
4,"I've been exploring this forum for awhile, and...",0


In [3]:
#clean the data for an LLM
import re

class TextCleaner():
    def __init__(self):
        pass
    
    def clean_text(self, text):
        #change the text to lower case
        text = text.lower()

        #delete special characters
        text = re.sub(r'<.*?>', '', text)

        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

        #split the text by spaces
        text = re.sub(r"\s+", " ", text).strip()
        return text

cleaner = TextCleaner()
df['cleaned_text'] = df['text'].apply(cleaner.clean_text)


In [4]:
df.head()

,text,label,cleaned_text
0,My father - all of my life - has shifted betwe...,0,my father all of my life has shifted between a...
1,I have health anxiety where I go to the doctor...,3,i have health anxiety where i go to the doctor...
2,I was thinking about the differences between B...,0,i was thinking about the differences between b...
3,Let me preface this by saying that I promise I...,2,let me preface this by saying that i promise i...
4,"I've been exploring this forum for awhile, and...",0,ive been exploring this forum for awhile and j...


In [5]:
print(len(df))

465051


In [6]:
import numpy as np
#sadly we have to drop data point for the 24 hour time frame
drop_indices = np.random.choice(df.index, 300000, replace=False)
df = df.drop(drop_indices)

In [7]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

label_encoder = preprocessing.LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'].tolist())

In [8]:
train_df, test_df = train_test_split(df, test_size=0.2)

In [9]:
from datasets import Dataset
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

def tokenize_data(examples):
    return tokenizer(examples["cleaned_text"], truncation=True)

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

c:\Users\kiera\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 33011/33011 [00:03<00:00, 10156.96 examples/s]


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding

# Load pre-trained DistilBERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

# Prepare data collator for padding sequences
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch"
)

# Define Trainer object for training the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the trained model
trainer.save_model('../../server/mental-health-classifier')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\kiera\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 23%|██▎       | 11396/49515 [29:52<1:51:35,  5.69it/s]